In [1]:
import pandas as pd
import numpy as np
import configparser
import pymysql
import requests
import re
from tqdm import tqdm
import configparser
import openai
import os
import json

In [2]:
config=configparser.ConfigParser()
config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [3]:
mydb= pymysql.connect(
host=config['iimjobs_db']['hostname'],
user=config['iimjobs_db']['username'],
password=config['iimjobs_db']['password'],
db=config['iimjobs_db']['db_name']
)

mycursor = mydb.cursor()

In [4]:
# mycursor = mydb.cursor()

 
query = "SELECT userid from usr_applied_jobs_info where jobid=1374896"

 # 1372184

mycursor.execute(query)

result = mycursor.fetchall()


In [5]:
udf = pd.DataFrame(result,columns=['userid'])

In [6]:
uid_list = udf.userid.values

In [7]:
uid_list

array([1286631, 2284789, 2499355, ..., 2410347, 2358390, 2349233])

In [8]:
from user_info import *

In [9]:
df = get_user_data_search_embed(uid_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1129/1129 [00:00<00:00, 94608.99it/s]


In [10]:
df.sample(5)

,id,resume,keywords,current_designation,user_experience,professional_info,education_info
398,2642122,"ASHOK KUMAR NAMACHIVAYAM\n+91 9566095310, +91 ...",Analysis Machine_Learning Python Data_Visualiz...,Data Scientist & Machine learning intern,5,"[{'id': 3991437, 'userId': None, 'designation'...","[{'id': 97538045, 'userId': 2642122, 'institut..."
3,2503379,"Soubhagya Soubhagya Kumar Sahoo\nBhubaneswar, ...",Analysis RISK Tableau XGBoost Risk_Management ...,Data Analyst,0,"[{'id': 3768690, 'userId': None, 'designation'...","[{'id': 97381002, 'userId': 2503379, 'institut..."
858,2665012,AKSHAY JAIN akshayjn10@gmail.com\n+91 84410785...,Client Designing Financial Analysis Marketing ...,,0,[],"[{'id': 97563870, 'userId': 2665012, 'institut..."
361,2662556,Certified Data scientist with 6 years f T expe...,Credit Analysis Scientist Data_Scientist Secur...,Rubixe intership,6,"[{'id': 4030820, 'userId': None, 'designation'...","[{'id': 97566662, 'userId': 2662556, 'institut..."
518,2714406,Raunak Gupta\n+91 82904 24358 | raunakgupta242...,Analysis Sales Excel Matplotlib BI GIT Data_En...,Data Analyst Intern,0,"[{'id': 4191569, 'userId': None, 'designation'...","[{'id': 97633627, 'userId': 2714406, 'institut..."


In [11]:
# df[['id','resume']].sample(10).values

In [12]:
# df

In [13]:
df['resume']=df['resume'].str.lower()
df['current_designation']=df['current_designation'].str.lower()

In [14]:
df.sample(5)

,id,resume,keywords,current_designation,user_experience,professional_info,education_info
1070,2646114,"positions of responsibility\nmale,23\n98775330...",Analytics BI Power_BI Client Profitability Rep...,consultant,3,"[{'id': 3997672, 'userId': None, 'designation'...","[{'id': 97542534, 'userId': 2646114, 'institut..."
196,2582415,"phone\n7976386296\naddress\ngurgaon, haryana ,...",Client Analytics Data_Quality Retention SAAS C...,software development engineer - 1,2,"[{'id': 3902730, 'userId': None, 'designation'...","[{'id': 97472032, 'userId': 2582415, 'institut..."
911,2476738,"yavatmal, india 445001\n7741897567\nkazitariqu...",Analysis Visualization Data_Analysis Journal S...,business analyst,0,"[{'id': 3736931, 'userId': None, 'designation'...","[{'id': 97359438, 'userId': 2476738, 'institut..."
870,2259279,akash dharmale\ndob: 25/02/1996 | e-mail: dhar...,Research Investment Financial Engineering Stoc...,business research associate,0,"[{'id': 3387256, 'userId': None, 'designation'...","[{'id': 94745361, 'userId': 2259279, 'institut..."
999,2623175,education\nyear degree/examination institution...,Strategy Real_Estate Research Analysis Consult...,,0,[],"[{'id': 97517001, 'userId': 2623175, 'institut..."


In [15]:
df1= df[['id','current_designation','user_experience','resume']]

In [16]:
# df1['resume']=df1['resume'].str.lower()

In [17]:
df1.sample(5)

,id,current_designation,user_experience,resume
786,2599295,data analyst,0,dev tripathi\nresult-driven professional; expl...
319,2375944,assistant systems engineer,2,skills\nprofile\ncontact\neducation\nexperienc...
229,2631239,product analyst,1,"anubhav tiwari\nhouse no- 402, u-50/59, sector..."
745,2456409,management trainee,0,position of resposibility\nnirman technologies...
1003,2781599,,0,abhishek pandey\n+91-7905269084 | abhishekpand...


In [18]:
documents = []
for x in df1:
    y = 'user id: '+df1['id'].values+', '+'user experience: '+df1['user_experience'].values+', '+'user designation: '+df1['current_designation'].values+', '+'user resume: '+df1['resume'].values 
    documents.append(y)

In [19]:
len(documents[0])

1113

In [20]:
documents[0][10]

'user id: 2053027, user experience: 2, user designation: product developer, user resume: preet singh\n+91-870-029-9709 ps8765329@gmail.com www.linkedin.com/in/preetsingh17/ https://github.com/rudopf\nsummary\nelectrical engineering student with software development experience in app development and product development.\nproject experience includes applications of software and hardware. seeking full-time position may 2020 in web devel-\nopment and pharmaceutical production.\neducation\nb.tech, electrical engineering graduating may 2020\nguru gobind singh inderprasth university, delhi, india 3.2/4.0 gpa\nhmr institute of technology\nrelevant coursework: data structures and programmable logic, advanced excel in business,software tools and sys-\ntems programming,probability and computer applications,machine learning and algorithmic game theory.\ntechnical skills\ndata analysis and statistics: python, tableau\ndesign and modeling tools: solidworks, labview, matlab, microsoft office\nprogram

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.vectorstores import FAISS


In [22]:
# documents = df1.values

In [23]:
# documents[90]

In [24]:
# documents[:, 0] = [{'id':int(value)} for value in documents[:, 0]]


In [25]:
# documents[0][90]

In [26]:
# len(documents)

In [27]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000,chunk_overlap=10)
splits = text_splitter.split_text(str(documents[0]))
print(len(splits), splits[0])

9 ['user id: 1286631, user experience: 5, user designation: product manager, user resume: a r a v i n d s u b r a m a n i a n g\n5e4, happy windows, kattupakkam, chennai-600056 a.sbmn1994@gmail.com (+91)-9751053481\neducational qualifications\ncourse name year of\ncompletion\ninstitute name aggregate\nb.tech (production\nengineering)\n2015 national institute of technology tiruchirappalli 7.97\n(cgpa)\nclass xii (cbse) 2011 jawahar senior secondary school, neyveli 85.6%\nclass x (cbse) 2009 jawahar higher secondary school, neyveli 93.4%\nprofessional experience\nrbl bank private limited, manager cards portfolio, gurugram may 2022 apr 2023\n managing p&amp;l ownership of the emi products of credit cards x-sell portfolio\n monitoring and improving key metrics of the business like penetration, conversion and income\n implemented digital transformation and increased the topline and bottomline of business\nindustrybuying, category owner, new delhi jun 2021 jul 2021\n managed seller on-boardi

In [28]:
len(splits)

9

In [29]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs={'device': 'cuda'})

In [30]:
db = FAISS.from_texts(documents[0], embeddings)


In [31]:
import langchain
from queue import Queue
from typing import Any
from langchain.llms.huggingface_text_gen_inference import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.schema import LLMResult
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts.prompt import PromptTemplate
from anyio.from_thread import start_blocking_portal #For model callback streaming
import re
import gradio as gr
langchain.debug=True 

In [32]:
# model = 'meta-llama/Llama-2-7b-chat-hf'
model = "mistralai/Mistral-7B-Instruct-v0.2"

In [33]:
from bs4 import BeautifulSoup

In [34]:
text = '''
<p><b>About the company:</b><br/><br/>Bizup is an early stage startup, driven by the vision to empower the 60 Mn small businesses (SMEs) in India with technology. We are building the first of its kind short video app for B2B (business to business) transactions - think of it like Tiktok for businesses. Aim is to build India's largest network of business users, with the plan to onboard 2 Mn+ users in next 12 months, largely from Tier 2+ cities. Founders are IIT/IIM/ISB alumni, with leadership experience at McKinsey, Paytm, Tata (linkedin.com/in/suparngoel, linkedin.com/in/anshul-singhal-21696810/). If you are someone looking to join in the 0 to 1 journey of building a rocketship, in a huge opportunity area, then reach out to us!<br/><br/><b>About the role:</b><br/><br/>- In the dynamic position of Business Analyst, you will play a pivotal role in driving user engagement and satisfaction for our app. Your responsibilities will encompass a diverse range of critical functions, making this role both exciting and impactful. Here's what you will be doing:<br/><br/>- Analyze user data to understand behaviour, preferences, and trends, using these insights to shape customer engagement strategies<br/><br/>- Monitor day-to-day operations and manage the team to ensure smooth functioning in compliance with company standards<br/><br/>- Collaborate closely with cross-functional teams, including business, product development, and content team, to ensure seamless communication and alignment in delivering a unified user experience<br/><br/>- Maintain user engagement dashboards and prepare reports and present findings to senior management.<br/><br/><b>What we are looking for?</b><br/><br/>- Any graduate, or similar<br/><br/>- Strong analytical skills and proficiency in using software and tools for data analysis like Excel<br/><br/>- Familiarity with communication platforms, CRMs, and a willingness to stay updated on emerging tech trends<br/><br/>- Proven experience in managing relationships across cross functional teams and team management or a similar role</p>
'''

# Use BeautifulSoup to remove HTML tags
soup = BeautifulSoup(text, "html.parser")
cleaned_text = soup.get_text()

print(cleaned_text)


About the company:Bizup is an early stage startup, driven by the vision to empower the 60 Mn small businesses (SMEs) in India with technology. We are building the first of its kind short video app for B2B (business to business) transactions - think of it like Tiktok for businesses. Aim is to build India's largest network of business users, with the plan to onboard 2 Mn+ users in next 12 months, largely from Tier 2+ cities. Founders are IIT/IIM/ISB alumni, with leadership experience at McKinsey, Paytm, Tata (linkedin.com/in/suparngoel, linkedin.com/in/anshul-singhal-21696810/). If you are someone looking to join in the 0 to 1 journey of building a rocketship, in a huge opportunity area, then reach out to us!About the role:- In the dynamic position of Business Analyst, you will play a pivotal role in driving user engagement and satisfaction for our app. Your responsibilities will encompass a diverse range of critical functions, making this role both exciting and impactful. Here's what y

In [35]:
from langchain.llms import VLLM

llm = VLLM(model=model,
           trust_remote_code=True,  
           max_new_tokens=512,
           top_k=20,
           top_p=0.95,
           temperature=0)

INFO 02-14 16:37:48 llm_engine.py:73] Initializing an LLM engine with config: model='mistralai/Mistral-7B-Instruct-v0.2', tokenizer='mistralai/Mistral-7B-Instruct-v0.2', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)


pytorch_model-00001-of-00003.bin:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

INFO 02-14 18:37:42 llm_engine.py:223] # GPU blocks: 868, # CPU blocks: 2048
INFO 02-14 18:37:45 model_runner.py:394] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-14 18:37:55 model_runner.py:437] Graph capturing finished in 9 secs.


In [37]:
template = """
<s>[INST]You are an AI Agent working for a job platform and your task is to help a recruiter 
Use the following pieces of context to answer the question. If no context provided, answer like a AI assistant.
{context}
Question: {question} [/INST]
"""

retriever = db.as_retriever(
        search_kwargs={"k": 10}
    )

In [38]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever,     
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    }
)

In [41]:
result = qa_chain({"query": "give me ids of those users who have graduated from an IIM ?"})
# print(result)

[chain/start] [1:chain:RetrievalQA] Entering Chain run with input:
{
  "query": "give me ids of those users who have graduated from an IIM ?"
}
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] Entering Chain run with input:
{
  "question": "give me ids of those users who have graduated from an IIM ?",
  "context": "user id: 1479400, user experience: 2, user designation: senior business architecture analyst intern, user resume: \n\nuser id: 2078347, user experience: 3, user designation: data scientist, user resume: \n\nuser id: 2609971, user experience: 1, user designation: project engineer, user resume: \n\nuser id: 2514872, user experience: 2, user designation: assistant manager, user resume: educational details :\nb.tech (2020) - cse\n80%, kurukshetra university\nssc-xii (2016)\n86.8%, cbse\nhsc-x (2014)\ncgpa-8\n\n\nuser id: 2574906, user expe

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.92s/it]

[llm/end] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain > 5:llm:VLLM] [1.93s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Based on the provided context, there is only one user (id: 2514872) who has mentioned attending the Indian Institute of Management (IIM) in their resume.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain > 4:chain:LLMChain] [1.93s] Exiting Chain run with output:
{
  "text": "Based on the provided context, there is only one user (id: 2514872) who has mentioned attending the Indian Institute of Management (IIM) in their resume."
}
[chain/end] [1:chain:RetrievalQA > 3:chain:StuffDocumentsChain] [1.93s] Exiting Chain run with output:
{
  "output_text": "Based on the provided context, there is only one user (id: 2514872) who has mentioned attending the Indian Institute of Managem

In [42]:
print(result['result'])

Based on the provided context, there is only one user (id: 2514872) who has mentioned attending the Indian Institute of Management (IIM) in their resume.


In [46]:
pattern = re.compile(r'\d+\.\s*([^\n]+)\s*-\s*(\d+)')

# Find all matches in the text
matches = pattern.findall(result['result'])

# Extract User IDs and print them
user_ids = [(name.strip(), id) for name, id in matches]

In [47]:
user_ids

[]

In [48]:
uid_list = []
for user_id in user_ids:
    print(f"User: {user_id[0]}, ID: {user_id[1]}")
    uid_list.append(user_id[1])

In [49]:
uid_list

[]

In [50]:
# user_id_pattern = r'User ID: (\d+)' and r'id: (\d+)' and r'ID: (\d+)' and r'ID is : (\d+)' and r"His ID is (\d+)" and r'ID: (\d+)' and r'Id: (\d+)' and r' ID: (\d+)'and r' id: (\d+)'and r'ID: (\d+)' and r' id: (\d+)'
# user_id_match = re.search(r'User ID: (\d+)', result['result'])
# user_id = user_id_match.group(1)

user_ids = re.findall(r'User ID: (\d+)' and r'ID (\d+)' and r'user id: (\d+)', result['result'])

# user_ids = re.findall(user_id_pattern, result['result'])

In [46]:
user_ids

['2553080', '2553080']

In [47]:
data = get_user_data_search_embed(user_ids)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 27060.03it/s]


In [48]:
data

,id,resume,keywords,current_designation,user_experience,professional_info,education_info
0,2553080,SRIJEN JHINGTA Email: srijenj@gmail.com Ph.no....,Research Retail Client Brand Analysis Marketin...,Consultant - Business Consulting PI,2,"[{'id': 4252712, 'userId': None, 'designation'...","[{'id': 97439491, 'userId': 2553080, 'institut..."
1,2553080,SRIJEN JHINGTA Email: srijenj@gmail.com Ph.no....,Research Retail Client Brand Analysis Marketin...,Consultant - Business Consulting PI,2,"[{'id': 4252712, 'userId': None, 'designation'...","[{'id': 97439491, 'userId': 2553080, 'institut..."


In [198]:
df1.loc[df1['id']=='2312967']

,id,current_designation,user_experience,resume


In [199]:
df1.loc[df1['id']=='4567890']

,id,current_designation,user_experience,resume


In [200]:
# data['resume'].values

In [86]:
# user_id_pattern

In [88]:
# df.shape

In [89]:
# df.loc[df['resume'].str.contains('gaming')]['resume'].values

In [ ]:
# give me all those user ids atleat 4 who have worked as analyst
# give all those user with their id who have worked as business development manager